# Working with Text, Merge, Join and Concatenate and Pivot Table

## Learning Outcomes

[Reference](http://pandas.pydata.org/pandas-docs/stable/text.html)


## Working with Text Data

* Splitting and Replacing Strings
* Indexing with .str
* Extracting Substrings
* Testing for Strings that Match or Contain a Pattern
* Creating Indicator Variables
* Method Summary

## Merge, join, and concatenate
[Reference](http://pandas.pydata.org/pandas-docs/stable/merging.html)
* Concatenating objects
* Database-style DataFrame joining/merging

## Reshaping and Pivot Tables
[Reference](http://pandas.pydata.org/pandas-docs/stable/reshaping.html)
* Reshaping by pivoting DataFrame objects
* Reshaping by stacking and unstacking
* Reshaping by Melt
* Combining with stats and GroupBy
* Pivot tables and cross-tabulations
* Tiling
* Computing indicator / dummy variables
* Factorizing values

In [2]:
import pandas as pd
import numpy as np
print("Pandas version : {}".format(pd.__version__))
print("Numpy version : {}".format(np.__version__))

Pandas version : 0.22.0
Numpy version : 1.14.3


# Working with Text Data

## `str`



[Method Summary](http://pandas.pydata.org/pandas-docs/stable/text.html#method-summary)

Series and Index are equipped with a set of string processing methods that make it
easy to operate on each element of the array. Perhaps most importantly, these methods exclude missing/NA values automatically. These are accessed via the `str` attribute and generally have names matching the equivalent (scalar) built-in string methods:

In [3]:
s = pd.Series(['A', 'B', 'C', 'Aaba', 'Baca', np.nan, 'CABA', 'dog', 'cat'])
s

0       A
1       B
2       C
3    Aaba
4    Baca
5     NaN
6    CABA
7     dog
8     cat
dtype: object

In [4]:
s.str.lower()

0       a
1       b
2       c
3    aaba
4    baca
5     NaN
6    caba
7     dog
8     cat
dtype: object

In [5]:
s.str.upper()

0       A
1       B
2       C
3    AABA
4    BACA
5     NaN
6    CABA
7     DOG
8     CAT
dtype: object

In [6]:
s.str.len()

0    1.0
1    1.0
2    1.0
3    4.0
4    4.0
5    NaN
6    4.0
7    3.0
8    3.0
dtype: float64

In [7]:
idx = pd.Index([' jack', 'jill ', ' jesse ', 'frank'])
idx

Index([' jack', 'jill ', ' jesse ', 'frank'], dtype='object')

In [8]:
idx.str.strip() # remove all extra whitespace

Index(['jack', 'jill', 'jesse', 'frank'], dtype='object')

In [9]:
idx.str.lstrip() # remove leading whitespace

Index(['jack', 'jill ', 'jesse ', 'frank'], dtype='object')

In [10]:
idx.str.rstrip() # remove trailing whitespace

Index([' jack', 'jill', ' jesse', 'frank'], dtype='object')

In [11]:
df = pd.DataFrame(np.random.randn(3, 2), columns=[' Column A ', ' Column B '],
                  index=range(3))
df

,Column A,Column B
0,0.316950,0.977802
1,-0.065060,-0.341090
2,1.153167,0.773220


In [12]:
df.columns 
# although the above print out does not show. There are leading and
# trailing whitespace as you can see below

Index([' Column A ', ' Column B '], dtype='object')

In [13]:
df.columns.str.strip()

Index(['Column A', 'Column B'], dtype='object')

In [14]:
df.columns.str.lower()

Index([' column a ', ' column b '], dtype='object')

These string methods can then be used to clean up the columns as needed. Here we are removing leading and trailing whitespaces, lowercasing all names, and replacing any remaining whitespaces with underscores:

In [15]:
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')
df

,column_a,column_b
0,0.316950,0.977802
1,-0.065060,-0.341090
2,1.153167,0.773220


In [16]:
df.columns

Index(['column_a', 'column_b'], dtype='object')

## Splitting and Replacing Strings

In [17]:
s2 = pd.Series(['a_b_c', 'c_d_e', np.nan, 'f_g_h'])
s2

0    a_b_c
1    c_d_e
2      NaN
3    f_g_h
dtype: object

In [18]:
s2.str.split('_')

0    [a, b, c]
1    [c, d, e]
2          NaN
3    [f, g, h]
dtype: object

Access elements by `get` or `str`

In [19]:
s2.str.split('_').str.get(1)

0      b
1      d
2    NaN
3      g
dtype: object

In [20]:
s2.str.split('_').str[1]

0      b
1      d
2    NaN
3      g
dtype: object

convert to DataFrame

In [21]:
s2.str.split('_', expand=True)

,0,1,2
0,a,b,c
1,c,d,e
2,NaN,NaN,NaN
3,f,g,h


In [22]:
s2.str.split('_', expand=True, n=1) # limit splits

,0,1
0,a,b_c
1,c,d_e
2,NaN,NaN
3,f,g_h


`rsplit` is similar to `split` except it works in the reverse direction, i.e., from the end of the string to the beginning of the string:

In [23]:
s2.str.rsplit('_', expand=True, n=1)

,0,1
0,a_b,c
1,c_d,e
2,NaN,NaN
3,f_g,h


## Indexing with `.str`


You can use `[]` notation to directly index by position locations. If you index past the end of the string, the result will be a `NaN`.

In [24]:
s = pd.Series(['A', 'B', 'C', 'Aaba', 'Baca', np.nan,
               'CABA', 'dog', 'cat'])

In [26]:
s

0       A
1       B
2       C
3    Aaba
4    Baca
5     NaN
6    CABA
7     dog
8     cat
dtype: object

In [27]:
s.str[0]

0      A
1      B
2      C
3      A
4      B
5    NaN
6      C
7      d
8      c
dtype: object

In [28]:
s.str[1]

0    NaN
1    NaN
2    NaN
3      a
4      a
5    NaN
6      A
7      o
8      a
dtype: object

## Extracting Substrings

### Extract first match in each subject (extract)



The extract method accepts a regular expression with at least one capture group.

Extracting a regular expression with more than one group returns a DataFrame with one column per group.

In [29]:
pd.Series(['a1', 'b2', 'c3']).str.extract('([ab])(\d)', expand=False)
# In regular expressions [ab] means find ab in the string
# \d means find digits

,0,1
0,a,1
1,b,2
2,NaN,NaN


Elements that do not match return a row filled with `NaN`. Thus, a Series of messy strings can be “converted” into a like-indexed Series or DataFrame of cleaned-up or more useful strings, without necessitating `get()` to access tuples or `re.match` objects. The dtype of the result is always object, even if no match is found and the result only contains `NaN`.

Example of using **Named groups**:

Instead of referring to them by numbers, groups can be referenced by a name.

The syntax for a named group is one of the Python-specific extensions: `(?P<name>...)`. name is, obviously, the name of the group. Named groups behave exactly like capturing groups, and additionally associate a name with a group.

In [30]:
pd.Series(['a1', 'b2', 'c3']).str.extract('(?P<letter>[ab])(?P<digit>\d)', expand=False)

,letter,digit
0,a,1
1,b,2
2,NaN,NaN


and optional groups like

In [31]:
pd.Series(['a1', 'b2', '3']).str.extract('([ab])?(\d)', expand=False)

,0,1
0,a,1
1,b,2
2,NaN,3


can also be used. Note that any capture group names in the regular expression will be used for column names; otherwise capture group numbers will be used.

Extracting a regular expression with one group returns a DataFrame with one column if expand=True.

Calling on an `Index` with a regex with exactly one capture group returns a `DataFrame` with one column if `expand=True`,

In [32]:
s = pd.Series(["a1", "b2", "c3"], ["A11", "B22", "C33"])
s

A11    a1
B22    b2
C33    c3
dtype: object

In [33]:
s.index

Index(['A11', 'B22', 'C33'], dtype='object')

In [34]:
s.index.str.extract("(?P<letter>[a-zA-Z])", expand=True)
# match lowercase and uppercase letters, [a-zA-Z]

,letter
0,A
1,B
2,C


In [35]:
s.index.str.extract("(?P<letter>[a-zA-Z])", expand=False)

Index(['A', 'B', 'C'], dtype='object', name='letter')

Calling on an `Index` with a regex with more than one capture group returns a `DataFrame` if `expand=True`.

In [36]:
s.index.str.extract("(?P<letter>[a-zA-Z])([0-9]+)", expand=True)
# match lowercase and uppercase letters, [a-zA-Z]. Name this group "Letter
# [0-9]+ = match Decimal form

,letter,1
0,A,11
1,B,22
2,C,33


### Extract all matches in each subject (extractall)

In [37]:
s = pd.Series(["a1a2", "b1", "c1"], ["A", "B", "C"])
s

A    a1a2
B      b1
C      c1
dtype: object

In [38]:
two_groups = '(?P<letter>[a-z])(?P<digit>[0-9])'
s.str.extract(two_groups, expand=True)    # extract returns only first match

,letter,digit
A,a,1
B,b,1
C,c,1


In [39]:
s.str.extractall(two_groups)

letter digit
  match             
A 0          a     1
  1          a     2
B 0          b     1
C 0          c     1

the `extractall` method returns every match. The result of `extractall` is always a `DataFrame` with a `MultiIndex` on its rows. The last level of the `MultiIndex` is named `match` and indicates the order in the subject.

## Testing for Strings that Match or Contain a Pattern


You can check whether elements contain a pattern:

In [40]:
pattern = r'[a-z][0-9]'
pd.Series(['1', '2', '3a', '3b', '03c']).str.contains(pattern)

0    False
1    False
2    False
3    False
4    False
dtype: bool

In [41]:
pd.Series(['10', '20', 'a3', '3a']).str.contains(pattern)

0    False
1    False
2     True
3    False
dtype: bool

In [43]:
pd.Series(['1', '2', '3a', '3b', '03c']).str.match(pattern)

0    False
1    False
2    False
3    False
4    False
dtype: bool

In [44]:
pd.Series(['10', '20', 'a3', '3a']).str.match(pattern)

0    False
1    False
2     True
3    False
dtype: bool

The distinction between `match` and `contains` is strictness: `match` relies on strict `re.match`, while `contains` relies on `re.search`.

For more, refer to the [online reference](https://docs.python.org/3/howto/regex.html#match-versus-search)

In [45]:
s4 = pd.Series(['A', 'B', 'C', 'Aaba', 'Baca', np.nan, 'CABA', 'dog', 'cat'])
s4.str.contains('A', na=False)

0     True
1    False
2    False
3     True
4    False
5    False
6     True
7    False
8    False
dtype: bool

## Creating Indicator Variables


You can extract dummy variables from string columns. For example if they are separated by a `'|'`:

In [46]:
s = pd.Series(['a', 'a|b', np.nan, 'a|c'])
s

0      a
1    a|b
2    NaN
3    a|c
dtype: object

In [47]:
s.str.get_dummies(sep='|')

,a,b,c
0,1,0,0
1,1,1,0
2,0,0,0
3,1,0,1


String Index also supports `get_dummies` which returns a `MultiIndex`.

In [48]:
idx = pd.Index(['a', 'a|b', np.nan, 'a|c'])
idx

Index(['a', 'a|b', nan, 'a|c'], dtype='object')

In [49]:
res = idx.str.get_dummies(sep='|')
res

MultiIndex(levels=[[0, 1], [0, 1], [0, 1]],
           labels=[[1, 1, 0, 1], [0, 1, 0, 0], [0, 0, 0, 1]],
           names=['a', 'b', 'c'])

****

# Merge, join, and concatenate 

## Concatenating objects

### Introduction

[reference](https://towardsdatascience.com/python-for-data-science-8-concepts-you-may-have-forgotten-i-did-825966908393)

Concat allows the user to append one or more dataframes to each other either below or next to it

In [50]:
df1 = pd.DataFrame({'A': ['A0', 'A1', 'A2', 'A3'],
                    'B': ['B0', 'B1', 'B2', 'B3'],
                    'C': ['C0', 'C1', 'C2', 'C3'],
                    'D': ['D0', 'D1', 'D2', 'D3']},
                   index=[0, 1, 2, 3])
df1

,A,B,C,D
0,A0,B0,C0,D0
1,A1,B1,C1,D1
2,A2,B2,C2,D2
3,A3,B3,C3,D3


In [51]:
df2 = pd.DataFrame({'A': ['A4', 'A5', 'A6', 'A7'],
                    'B': ['B4', 'B5', 'B6', 'B7'],
                    'C': ['C4', 'C5', 'C6', 'C7'],
                    'D': ['D4', 'D5', 'D6', 'D7']},
                   index=[4, 5, 6, 7])
df2

,A,B,C,D
4,A4,B4,C4,D4
5,A5,B5,C5,D5
6,A6,B6,C6,D6
7,A7,B7,C7,D7


In [52]:
df3 = pd.DataFrame({'A': ['A8', 'A9', 'A10', 'A11'],
                    'B': ['B8', 'B9', 'B10', 'B11'],
                    'C': ['C8', 'C9', 'C10', 'C11'],
                    'D': ['D8', 'D9', 'D10', 'D11']},
                   index=[8, 9, 10, 11])
df3

,A,B,C,D
8,A8,B8,C8,D8
9,A9,B9,C9,D9
10,A10,B10,C10,D10
11,A11,B11,C11,D11


In [53]:
frames = [df1, df2, df3]
result = pd.concat(frames)
result

,A,B,C,D
0,A0,B0,C0,D0
1,A1,B1,C1,D1
2,A2,B2,C2,D2
3,A3,B3,C3,D3
4,A4,B4,C4,D4
5,A5,B5,C5,D5
6,A6,B6,C6,D6
7,A7,B7,C7,D7
8,A8,B8,C8,D8
9,A9,B9,C9,D9


Suppose we wanted to associate specific keys with each of the pieces of the chopped up DataFrame. We can do this using the `keys` argument:

In [54]:
result = pd.concat(frames, keys=['x', 'y', 'z'])
result

A    B    C    D
x 0    A0   B0   C0   D0
  1    A1   B1   C1   D1
  2    A2   B2   C2   D2
  3    A3   B3   C3   D3
y 4    A4   B4   C4   D4
  5    A5   B5   C5   D5
  6    A6   B6   C6   D6
  7    A7   B7   C7   D7
z 8    A8   B8   C8   D8
  9    A9   B9   C9   D9
  10  A10  B10  C10  D10
  11  A11  B11  C11  D11

The resulting object’s index has a **hierarchical index**. This means that we can now do stuff like select out each chunk by key:

In [56]:
result.loc['y']

,A,B,C,D
4,A4,B4,C4,D4
5,A5,B5,C5,D5
6,A6,B6,C6,D6
7,A7,B7,C7,D7


Note It is worth noting however, that `concat` (and `append`) makes a full copy of the data, and that constantly reusing this function can create a significant performance hit. If you need to use the operation over several datasets, use a list comprehension.

`frames = [process_your_file(f) for f in files]`

`result = pd.concat(frames)`

### Set logic on the other axes



When gluing together multiple DataFrames, for example, you have a choice of how to handle the other axes (other than the one being concatenated). This can be done in three ways:
* Take the (sorted) **union** of them all, `join='outer'`. This is the default option as it results in zero information loss.
* Take the **intersection**, `join='inner'`.
* Use a specific index (in the case of DataFrame), i.e. the `join_axes` argument

Here is a example of each of these methods. First, the default `join='outer'` behavior:

In [57]:
df4 = pd.DataFrame({'B': ['B2', 'B3', 'B6', 'B7'],
                    'D': ['D2', 'D3', 'D6', 'D7'],
                    'F': ['F2', 'F3', 'F6', 'F7']},
                   index=[2, 3, 6, 7])
df4

,B,D,F
2,B2,D2,F2
3,B3,D3,F3
6,B6,D6,F6
7,B7,D7,F7


In [58]:
df1

,A,B,C,D
0,A0,B0,C0,D0
1,A1,B1,C1,D1
2,A2,B2,C2,D2
3,A3,B3,C3,D3


In [60]:
result = pd.concat([df1, df4], axis=0)
# join on index since axis=0
result

,A,B,C,D,F
0,A0,B0,C0,D0,NaN
1,A1,B1,C1,D1,NaN
2,A2,B2,C2,D2,NaN
3,A3,B3,C3,D3,NaN
2,NaN,B2,NaN,D2,F2
3,NaN,B3,NaN,D3,F3
6,NaN,B6,NaN,D6,F6
7,NaN,B7,NaN,D7,F7


In [59]:
result = pd.concat([df1, df4], axis=1)
# join on columns since axis=1
# columns B and D in this case
result

,A,B,C,D,B,D,F
0,A0,B0,C0,D0,NaN,NaN,NaN
1,A1,B1,C1,D1,NaN,NaN,NaN
2,A2,B2,C2,D2,B2,D2,F2
3,A3,B3,C3,D3,B3,D3,F3
6,NaN,NaN,NaN,NaN,B6,D6,F6
7,NaN,NaN,NaN,NaN,B7,D7,F7


Note that the row indexes have been unioned and sorted. 

Here is the same thing with `join='inner'`:

In [61]:
result = pd.concat([df1, df4], axis=1, join='inner')
# inner = intersection
# Column B and D, the intersection value are
# B: B2 and B3
# D: D2 and D3
result

,A,B,C,D,B,D,F
2,A2,B2,C2,D2,B2,D2,F2
3,A3,B3,C3,D3,B3,D3,F3


Lastly, suppose we just wanted to reuse the **exact index** from the original DataFrame:

In [62]:
result = pd.concat([df1, df4], axis=1, join_axes=[df1.index])
result

,A,B,C,D,B,D,F
0,A0,B0,C0,D0,NaN,NaN,NaN
1,A1,B1,C1,D1,NaN,NaN,NaN
2,A2,B2,C2,D2,B2,D2,F2
3,A3,B3,C3,D3,B3,D3,F3


### Concatenating using append


A useful shortcut to `concat` are the `append` instance methods on Series and DataFrame. These methods actually predated `concat`. They concatenate along `axis=0`, namely the index:

In [63]:
df1

,A,B,C,D
0,A0,B0,C0,D0
1,A1,B1,C1,D1
2,A2,B2,C2,D2
3,A3,B3,C3,D3


In [64]:
df2

,A,B,C,D
4,A4,B4,C4,D4
5,A5,B5,C5,D5
6,A6,B6,C6,D6
7,A7,B7,C7,D7


In [65]:
result = df1.append(df2)
result

,A,B,C,D
0,A0,B0,C0,D0
1,A1,B1,C1,D1
2,A2,B2,C2,D2
3,A3,B3,C3,D3
4,A4,B4,C4,D4
5,A5,B5,C5,D5
6,A6,B6,C6,D6
7,A7,B7,C7,D7


In the case of DataFrame, the indexes must be disjoint, continuous without joining, but the columns do not need to be:

In [66]:
df1

,A,B,C,D
0,A0,B0,C0,D0
1,A1,B1,C1,D1
2,A2,B2,C2,D2
3,A3,B3,C3,D3


In [67]:
df4

,B,D,F
2,B2,D2,F2
3,B3,D3,F3
6,B6,D6,F6
7,B7,D7,F7


In [68]:
result = df1.append(df4)
result

,A,B,C,D,F
0,A0,B0,C0,D0,NaN
1,A1,B1,C1,D1,NaN
2,A2,B2,C2,D2,NaN
3,A3,B3,C3,D3,NaN
2,NaN,B2,NaN,D2,F2
3,NaN,B3,NaN,D3,F3
6,NaN,B6,NaN,D6,F6
7,NaN,B7,NaN,D7,F7


`append` may take multiple objects to concatenate:

In [69]:
df1

,A,B,C,D
0,A0,B0,C0,D0
1,A1,B1,C1,D1
2,A2,B2,C2,D2
3,A3,B3,C3,D3


In [70]:
df2

,A,B,C,D
4,A4,B4,C4,D4
5,A5,B5,C5,D5
6,A6,B6,C6,D6
7,A7,B7,C7,D7


In [71]:
df3

,A,B,C,D
8,A8,B8,C8,D8
9,A9,B9,C9,D9
10,A10,B10,C10,D10
11,A11,B11,C11,D11


In [72]:
result = df1.append([df2, df3])
result

,A,B,C,D
0,A0,B0,C0,D0
1,A1,B1,C1,D1
2,A2,B2,C2,D2
3,A3,B3,C3,D3
4,A4,B4,C4,D4
5,A5,B5,C5,D5
6,A6,B6,C6,D6
7,A7,B7,C7,D7
8,A8,B8,C8,D8
9,A9,B9,C9,D9


Note Unlike **list.append** method, which appends to the original list and returns nothing, `append` here does not modify `df1` and returns its copy with `df2` appended.

### Ignoring indexes on the concatenation axis


For DataFrames which don’t have a meaningful index, you may wish to append them and ignore the fact that they may have overlapping indexes:

To do this, use the `ignore_index` argument:

In [73]:
df1

,A,B,C,D
0,A0,B0,C0,D0
1,A1,B1,C1,D1
2,A2,B2,C2,D2
3,A3,B3,C3,D3


In [74]:
df4

,B,D,F
2,B2,D2,F2
3,B3,D3,F3
6,B6,D6,F6
7,B7,D7,F7


In [75]:
result = pd.concat([df1, df4], ignore_index=True)
result

,A,B,C,D,F
0,A0,B0,C0,D0,NaN
1,A1,B1,C1,D1,NaN
2,A2,B2,C2,D2,NaN
3,A3,B3,C3,D3,NaN
4,NaN,B2,NaN,D2,F2
5,NaN,B3,NaN,D3,F3
6,NaN,B6,NaN,D6,F6
7,NaN,B7,NaN,D7,F7


### Concatenating with mixed ndims


You can concatenate a mix of Series and DataFrames. The Series will be transformed to DataFrames with the column name as the name of the Series.

In [76]:
s1 = pd.Series(['X0', 'X1', 'X2', 'X3'], name='X')
s1

0    X0
1    X1
2    X2
3    X3
Name: X, dtype: object

In [77]:
df1

,A,B,C,D
0,A0,B0,C0,D0
1,A1,B1,C1,D1
2,A2,B2,C2,D2
3,A3,B3,C3,D3


In [78]:
result = pd.concat([df1, s1], axis=1)
result

,A,B,C,D,X
0,A0,B0,C0,D0,X0
1,A1,B1,C1,D1,X1
2,A2,B2,C2,D2,X2
3,A3,B3,C3,D3,X3


### More concatenating with group keys


A fairly common use of the `keys` argument is to override the column names when creating a new DataFrame based on existing Series. Notice how the default behaviour consists on letting the resulting DataFrame inherits the parent Series’ name, when these existed.

In [79]:
s3 = pd.Series([0, 1, 2, 3], name='foo')
s4 = pd.Series([0, 1, 2, 3])
s5 = pd.Series([0, 1, 4, 5])
pd.concat([s3, s4, s5], axis=1)

,foo,0,1
0,0,0,0
1,1,1,1
2,2,2,4
3,3,3,5


Through the `keys` argument we can override the existing column names.

In [80]:
pd.concat([s3, s4, s5], axis=1, keys=['red','blue','yellow'])

,red,blue,yellow
0,0,0,0
1,1,1,1
2,2,2,4
3,3,3,5


Let’s consider now a variation on the very first example presented:

In [81]:
result = pd.concat(frames, keys=['x', 'y', 'z']) # difference is axis=0 here
result

A    B    C    D
x 0    A0   B0   C0   D0
  1    A1   B1   C1   D1
  2    A2   B2   C2   D2
  3    A3   B3   C3   D3
y 4    A4   B4   C4   D4
  5    A5   B5   C5   D5
  6    A6   B6   C6   D6
  7    A7   B7   C7   D7
z 8    A8   B8   C8   D8
  9    A9   B9   C9   D9
  10  A10  B10  C10  D10
  11  A11  B11  C11  D11

#### Specify your own key using 'dict'


You can also pass a `dict` to concat in which case the dict `keys` will be used for the keys argument (unless other keys are specified):

In [82]:
df1

,A,B,C,D
0,A0,B0,C0,D0
1,A1,B1,C1,D1
2,A2,B2,C2,D2
3,A3,B3,C3,D3


In [83]:
df2

,A,B,C,D
4,A4,B4,C4,D4
5,A5,B5,C5,D5
6,A6,B6,C6,D6
7,A7,B7,C7,D7


In [84]:
df3

,A,B,C,D
8,A8,B8,C8,D8
9,A9,B9,C9,D9
10,A10,B10,C10,D10
11,A11,B11,C11,D11


In [85]:
pieces = {'x': df1, 'y': df2, 'z': df3}
result = pd.concat(pieces)
result

A    B    C    D
x 0    A0   B0   C0   D0
  1    A1   B1   C1   D1
  2    A2   B2   C2   D2
  3    A3   B3   C3   D3
y 4    A4   B4   C4   D4
  5    A5   B5   C5   D5
  6    A6   B6   C6   D6
  7    A7   B7   C7   D7
z 8    A8   B8   C8   D8
  9    A9   B9   C9   D9
  10  A10  B10  C10  D10
  11  A11  B11  C11  D11

In [86]:
result = pd.concat(pieces, keys=['z', 'y'])
result

A    B    C    D
z 8    A8   B8   C8   D8
  9    A9   B9   C9   D9
  10  A10  B10  C10  D10
  11  A11  B11  C11  D11
y 4    A4   B4   C4   D4
  5    A5   B5   C5   D5
  6    A6   B6   C6   D6
  7    A7   B7   C7   D7

#### Levels


The MultiIndex created has **levels** that are constructed from the passed keys and the index of the DataFrame pieces:

In [87]:
result.index.levels

FrozenList([['z', 'y'], [4, 5, 6, 7, 8, 9, 10, 11]])

If you wish to specify other levels (as will occasionally be the case), you can do so using the `levels` argument:

In [88]:
result = pd.concat(pieces, keys=['x', 'y', 'z'],
                   levels=[['z', 'y', 'x', 'w']],
                   names=['group_key'])
result

A    B    C    D
group_key                       
x         0    A0   B0   C0   D0
          1    A1   B1   C1   D1
          2    A2   B2   C2   D2
          3    A3   B3   C3   D3
y         4    A4   B4   C4   D4
          5    A5   B5   C5   D5
          6    A6   B6   C6   D6
          7    A7   B7   C7   D7
z         8    A8   B8   C8   D8
          9    A9   B9   C9   D9
          10  A10  B10  C10  D10
          11  A11  B11  C11  D11

In [89]:
result.index.levels

FrozenList([['z', 'y', 'x', 'w'], [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]])

Yes, this is fairly esoteric, but is actually necessary for implementing things like GroupBy where the order of a categorical variable is meaningful.

### Appending rows to a DataFrame


While not especially efficient (since a new object must be created), you can append a single row to a DataFrame by passing a Series or dict to `append`, which returns a new DataFrame as above.

In [90]:
df1

,A,B,C,D
0,A0,B0,C0,D0
1,A1,B1,C1,D1
2,A2,B2,C2,D2
3,A3,B3,C3,D3


In [91]:
s2 = pd.Series(['X0', 'X1', 'X2', 'X3'], index=['A', 'B', 'C', 'D'])
s2

A    X0
B    X1
C    X2
D    X3
dtype: object

In [92]:
result = df1.append(s2, ignore_index=True)
result

,A,B,C,D
0,A0,B0,C0,D0
1,A1,B1,C1,D1
2,A2,B2,C2,D2
3,A3,B3,C3,D3
4,X0,X1,X2,X3


You should use `ignore_index` with this method to instruct DataFrame to discard its index. If you wish to preserve the index, you should construct an appropriately-indexed DataFrame and append or concatenate those objects.

You can also pass a list of dicts or Series:

In [93]:
dicts = [{'A': 1, 'B': 2, 'C': 3, 'X': 4},
         {'A': 5, 'B': 6, 'C': 7, 'Y': 8}]

In [94]:
result = df1.append(dicts, ignore_index=True)
result

,A,B,C,D,X,Y
0,A0,B0,C0,D0,NaN,NaN
1,A1,B1,C1,D1,NaN,NaN
2,A2,B2,C2,D2,NaN,NaN
3,A3,B3,C3,D3,NaN,NaN
4,1,2,3,NaN,4.0,NaN
5,5,6,7,NaN,NaN,8.0


****

## Database-style DataFrame joining/merging

### Merging

Merge combines multiple dataframes on specific, common columns that serve as the primary key.

Check out the [cookbook](http://pandas.pydata.org/pandas-docs/stable/cookbook.html#cookbook-merge) as well.
pandas provides a single function, `merge`, as the entry point for all standard database join operations between DataFrame objects:

In [95]:
# merge(left, right, how='inner', on=None, left_on=None, right_on=None,
#       left_index=False, right_index=False, sort=True,
#       suffixes=('_x', '_y'), copy=True, indicator=False)

* `left`: A DataFrame object
* `right`: Another DataFrame object
* `on`: Columns (names) to join on. Must be found in both the left and right DataFrame objects. If not passed and `left_index` and `right_index` are False, the intersection of the columns in the DataFrames will be inferred to be the join keys
* `left_on`: Columns from the left DataFrame to use as keys. Can either be column names or arrays with length equal to the length of the DataFrame
* `right_on`: Columns from the right DataFrame to use as keys. Can either be column names or arrays with length equal to the length of the DataFrame
* `left_index`: If True, use the index (row labels) from the left DataFrame as its join key(s). In the case of a DataFrame with a MultiIndex (hierarchical), the number of levels must match the number of join keys from the right DataFrame
* `right_index`: Same usage as `left_index` for the right DataFrame
* `how`: One of `'left'`, `'right'`, `'outer'`, `'inner'`. Defaults to `inner`. See below for more detailed description of each method
* `sort`: Sort the result DataFrame by the join keys in lexicographical order. Defaults to True, setting to False will improve performance substantially in many cases
* `suffixes`: A tuple of string suffixes to apply to overlapping columns. Defaults to (`'_x', '_y'`).
* `copy`: Always copy data (default `True`) from the passed DataFrame objects, even when reindexing is not necessary. Cannot be avoided in many cases but may improve performance / memory usage. The cases where copying can be avoided are somewhat pathological but this option is provided nonetheless.
* `indicator`: Add a column to the output DataFrame called `_merge` with information on the source of each row. `_merge` is Categorical-type and takes on a value of `left_only` for observations whose merge key only appears in `'left'` DataFrame, `right_only` for observations whose merge key only appears in `'right'` DataFrame, and `both` if the observation’s merge key is found in both.

The related `DataFrame.join` method, uses `merge` internally for the index-on-index and index-on-column(s) joins, but **joins on indexes** by default rather than trying to **join on common columns** (the default behavior for **merge**). If you are joining on index, you may wish to use `DataFrame.join` to save yourself some typing.

### Brief primer on merge methods (relational algebra)


There are several cases to consider which are very important to understand:
* **one-to-one joins**: for example when joining two DataFrame objects on their indexes (which must contain unique values)
* **many-to-one joins**: for example when joining an index (unique) to one or more columns in a DataFrame
* **many-to-many joins**: joining columns on columns.

Note: When joining columns on columns (potentially a many-to-many join), any indexes on the passed DataFrame objects **will be discarded**.

It is worth spending some time understanding the result of the **many-to-many** join case. In SQL / standard relational algebra, if a key combination appears more than once in both tables, the resulting table will have the **Cartesian product** of the associated data. Here is a very basic example with one unique key combination:

In [96]:
left = pd.DataFrame({'key': ['K0', 'K1', 'K2', 'K3'],
                     'A': ['A0', 'A1', 'A2', 'A3'],
                     'B': ['B0', 'B1', 'B2', 'B3']})
left

,A,B,key
0,A0,B0,K0
1,A1,B1,K1
2,A2,B2,K2
3,A3,B3,K3


In [97]:
right = pd.DataFrame({'key': ['K0', 'K1', 'K2', 'K3'],
                      'C': ['C0', 'C1', 'C2', 'C3'],
                      'D': ['D0', 'D1', 'D2', 'D3']})
right

,C,D,key
0,C0,D0,K0
1,C1,D1,K1
2,C2,D2,K2
3,C3,D3,K3


In [98]:
result = pd.merge(left, right, on='key')
result

,A,B,key,C,D
0,A0,B0,K0,C0,D0
1,A1,B1,K1,C1,D1
2,A2,B2,K2,C2,D2
3,A3,B3,K3,C3,D3


Example with multiple join keys:

In [99]:
left = pd.DataFrame({'key1': ['K0', 'K0', 'K1', 'K2'],
                     'key2': ['K0', 'K1', 'K0', 'K1'],
                     'A': ['A0', 'A1', 'A2', 'A3'],
                     'B': ['B0', 'B1', 'B2', 'B3']})
left

,A,B,key1,key2
0,A0,B0,K0,K0
1,A1,B1,K0,K1
2,A2,B2,K1,K0
3,A3,B3,K2,K1


In [100]:
right = pd.DataFrame({'key1': ['K0', 'K1', 'K1', 'K2'],
                      'key2': ['K0', 'K0', 'K0', 'K0'],
                      'C': ['C0', 'C1', 'C2', 'C3'],
                      'D': ['D0', 'D1', 'D2', 'D3']})
right

,C,D,key1,key2
0,C0,D0,K0,K0
1,C1,D1,K1,K0
2,C2,D2,K1,K0
3,C3,D3,K2,K0


In [101]:
result = pd.merge(left, right, on=['key1', 'key2'])
result

,A,B,key1,key2,C,D
0,A0,B0,K0,K0,C0,D0
1,A2,B2,K1,K0,C1,D1
2,A2,B2,K1,K0,C2,D2


The `how` argument to `merge` specifies how to determine which keys are to be included in the resulting table. If a key combination **does not appear** in either the left or right tables, the values in the joined table will be `NA`. Here is a summary of the `how` options and their SQL equivalent names:

 | Merge method	 | SQL Join Name | Description | 
  | -  |  -  | - | 
 | left	 | LEFT OUTER JOIN | Use keys from left frame only | 
 | right | RIGHT OUTER JOIN | Use keys from right frame only | 
 | outer | FULL OUTER JOIN | Use union of keys from both frames | 
 | inner | INNER JOIN | Use intersection of keys from both frames | 

**how='left'**

In [102]:
result = pd.merge(left, right, how='left', on=['key1', 'key2'])
result
# note with how='left', we keep all of the left dataframe

,A,B,key1,key2,C,D
0,A0,B0,K0,K0,C0,D0
1,A1,B1,K0,K1,NaN,NaN
2,A2,B2,K1,K0,C1,D1
3,A2,B2,K1,K0,C2,D2
4,A3,B3,K2,K1,NaN,NaN


**how='right'**

In [103]:
result = pd.merge(left, right, how='right', on=['key1', 'key2'])
result

,A,B,key1,key2,C,D
0,A0,B0,K0,K0,C0,D0
1,A2,B2,K1,K0,C1,D1
2,A2,B2,K1,K0,C2,D2
3,NaN,NaN,K2,K0,C3,D3


**how='outer'**

In [104]:
result = pd.merge(left, right, how='outer', on=['key1', 'key2'])
result

,A,B,key1,key2,C,D
0,A0,B0,K0,K0,C0,D0
1,A1,B1,K0,K1,NaN,NaN
2,A2,B2,K1,K0,C1,D1
3,A2,B2,K1,K0,C2,D2
4,A3,B3,K2,K1,NaN,NaN
5,NaN,NaN,K2,K0,C3,D3


**how='inner'**

In [105]:
result = pd.merge(left, right, how='inner', on=['key1', 'key2'])
result

,A,B,key1,key2,C,D
0,A0,B0,K0,K0,C0,D0
1,A2,B2,K1,K0,C1,D1
2,A2,B2,K1,K0,C2,D2


### The merge indicator

In [106]:
df1 = pd.DataFrame({'col1':[0,1], 'col_left':['a','b']})
df1

,col1,col_left
0,0,a
1,1,b


In [107]:
df2 = pd.DataFrame({'col1':[1,2,2],'col_right':[2,2,2]})
df2

,col1,col_right
0,1,2
1,2,2
2,2,2


In [108]:
pd.merge(df1, df2, on='col1', how='outer', indicator=True)

,col1,col_left,col_right,_merge
0,0,a,NaN,left_only
1,1,b,2.0,both
2,2,NaN,2.0,right_only
3,2,NaN,2.0,right_only


### Joining on **index**

Join, much like merge, combines two dataframes. However, it joins them based on their indices, rather than some specified column.


DataFrame.join is a convenient method for combining the columns of two potentially differently-indexed DataFrames into a single result DataFrame. 

In [109]:
left = pd.DataFrame({'A': ['A0', 'A1', 'A2'],
                     'B': ['B0', 'B1', 'B2']},
                    index=['K0', 'K1', 'K2'])
left

,A,B
K0,A0,B0
K1,A1,B1
K2,A2,B2


In [110]:
right = pd.DataFrame({'C': ['C0', 'C2', 'C3'],
                      'D': ['D0', 'D2', 'D3']},
                     index=['K0', 'K2', 'K3'])
right

,C,D
K0,C0,D0
K2,C2,D2
K3,C3,D3


In [111]:
result = left.join(right)
result

,A,B,C,D
K0,A0,B0,C0,D0
K1,A1,B1,NaN,NaN
K2,A2,B2,C2,D2


In [112]:
result = left.join(right, how='outer')
result

,A,B,C,D
K0,A0,B0,C0,D0
K1,A1,B1,NaN,NaN
K2,A2,B2,C2,D2
K3,NaN,NaN,C3,D3


In [113]:
result = left.join(right, how='inner')
result

,A,B,C,D
K0,A0,B0,C0,D0
K2,A2,B2,C2,D2


The data alignment here is on the indexes (row labels). This same behavior can be achieved using merge plus additional arguments instructing it to use the indexes:

In [114]:
result = pd.merge(left, right, left_index=True, right_index=True, how='outer')
result

,A,B,C,D
K0,A0,B0,C0,D0
K1,A1,B1,NaN,NaN
K2,A2,B2,C2,D2
K3,NaN,NaN,C3,D3


In [115]:
result = pd.merge(left, right, left_index=True, right_index=True, how='inner');
result

,A,B,C,D
K0,A0,B0,C0,D0
K2,A2,B2,C2,D2


### Joining key columns on an index


`join` takes an optional `on` argument which may be a column or multiple column names, which specifies that the passed DataFrame is to be aligned on that column in the DataFrame. These two function calls are completely equivalent:

In [116]:
left = pd.DataFrame({'A': ['A0', 'A1', 'A2', 'A3'],
                     'B': ['B0', 'B1', 'B2', 'B3'],
                     'key': ['K0', 'K1', 'K0', 'K1']})
left

,A,B,key
0,A0,B0,K0
1,A1,B1,K1
2,A2,B2,K0
3,A3,B3,K1


In [117]:
right = pd.DataFrame({'C': ['C0', 'C1'],
                      'D': ['D0', 'D1']},
                     index=['K0', 'K1'])
right

,C,D
K0,C0,D0
K1,C1,D1


In [118]:
result = left.join(right, on='key')
result

,A,B,key,C,D
0,A0,B0,K0,C0,D0
1,A1,B1,K1,C1,D1
2,A2,B2,K0,C0,D0
3,A3,B3,K1,C1,D1


In [119]:
result = pd.merge(left, right, left_on='key', right_index=True,
                  how='left', sort=False);
result

,A,B,key,C,D
0,A0,B0,K0,C0,D0
1,A1,B1,K1,C1,D1
2,A2,B2,K0,C0,D0
3,A3,B3,K1,C1,D1


To join on multiple keys, the passed DataFrame must have a MultiIndex:

In [120]:
left = pd.DataFrame({'A': ['A0', 'A1', 'A2', 'A3'],
                     'B': ['B0', 'B1', 'B2', 'B3'],
                     'key1': ['K0', 'K0', 'K1', 'K2'],
                     'key2': ['K0', 'K1', 'K0', 'K1']})
left

,A,B,key1,key2
0,A0,B0,K0,K0
1,A1,B1,K0,K1
2,A2,B2,K1,K0
3,A3,B3,K2,K1


In [121]:
index = pd.MultiIndex.from_tuples([('K0', 'K0'), ('K1', 'K0'),
                                   ('K2', 'K0'), ('K2', 'K1')])
index

MultiIndex(levels=[['K0', 'K1', 'K2'], ['K0', 'K1']],
           labels=[[0, 1, 2, 2], [0, 0, 0, 1]])

In [122]:
right = pd.DataFrame({'C': ['C0', 'C1', 'C2', 'C3'],
                      'D': ['D0', 'D1', 'D2', 'D3']},
                     index=index)
right

C   D
K0 K0  C0  D0
K1 K0  C1  D1
K2 K0  C2  D2
   K1  C3  D3

In [123]:
result = left.join(right, on=['key1', 'key2'])
result

,A,B,key1,key2,C,D
0,A0,B0,K0,K0,C0,D0
1,A1,B1,K0,K1,NaN,NaN
2,A2,B2,K1,K0,C1,D1
3,A3,B3,K2,K1,C3,D3


The default for `DataFrame.join` is to perform a left join (essentially a “VLOOKUP” operation, for Excel users), which uses only the keys found in the calling DataFrame. Other join types, for example inner join, can be just as easily performed:

In [124]:
result = left.join(right, on=['key1', 'key2'], how='inner')
result

,A,B,key1,key2,C,D
0,A0,B0,K0,K0,C0,D0
2,A2,B2,K1,K0,C1,D1
3,A3,B3,K2,K1,C3,D3


As you can see, this drops any rows where there was no match.

***********

# Reshaping and Pivot Tables

### Introduction


[online reference](http://pandas.pydata.org/pandas-docs/stable/reshaping.ht  * ml)

Topics:
* Reshaping by pivoting DataFrame objects
* Reshaping by stacking and unstacking
  * Multiple Levels
  * Missing Data
  * With a MultiIndex
* Reshaping by Melt
* Combining with stats and GroupBy
* Pivot tables
  * Adding margins
* Cross tabulations
  * Normalization
  * Adding Margins
* Tiling
* Computing indicator / dummy variables
* Factorizing values

### Reshaping by pivoting DataFrame objects

In [134]:
import pandas.util.testing as tm; tm.N = 3
def unpivot(frame):
    N, K = frame.shape
    data = {'value' : frame.values.ravel('F'),
            'variable' : np.asarray(frame.columns).repeat(N),
            'date' : np.tile(np.asarray(frame.index), K)}
    return pd.DataFrame(data, columns=['date', 'variable', 'value'])
df = unpivot(tm.makeTimeDataFrame())
df

,date,variable,value
0,2000-01-03,A,-1.799869
1,2000-01-04,A,1.011323
2,2000-01-05,A,-0.500127
3,2000-01-03,B,2.488754
4,2000-01-04,B,-0.864634
5,2000-01-05,B,-0.931677
6,2000-01-03,C,1.042697
7,2000-01-04,C,0.413949
8,2000-01-05,C,-0.976139
9,2000-01-03,D,-1.158521


In [135]:
# To select out everything for variable A we could do:
df[df['variable'] == 'A']

,date,variable,value
0,2000-01-03,A,-1.799869
1,2000-01-04,A,1.011323
2,2000-01-05,A,-0.500127


But suppose we wish to do time series operations with the variables. A better representation would be where the `columns` are the unique variables and an `index` of dates identifies individual observations. To reshape the data into this form, use the `pivot` function:

In [136]:
df.pivot(index='date', columns='variable', values='value')

variable,A,B,C,D
date,,,,
2000-01-03,-1.799869,2.488754,1.042697,-1.158521
2000-01-04,1.011323,-0.864634,0.413949,-0.043574
2000-01-05,-0.500127,-0.931677,-0.976139,-0.000943


If the `values` argument is omitted, and the input DataFrame has more than one column of values which are not used as column or index inputs to `pivot`, then the resulting “pivoted” DataFrame will have **hierarchical columns** whose topmost level indicates the respective value column:

In [137]:
df['value2'] = df['value'] * 2
df

,date,variable,value,value2
0,2000-01-03,A,-1.799869,-3.599738
1,2000-01-04,A,1.011323,2.022646
2,2000-01-05,A,-0.500127,-1.000253
3,2000-01-03,B,2.488754,4.977507
4,2000-01-04,B,-0.864634,-1.729269
5,2000-01-05,B,-0.931677,-1.863354
6,2000-01-03,C,1.042697,2.085394
7,2000-01-04,C,0.413949,0.827897
8,2000-01-05,C,-0.976139,-1.952279
9,2000-01-03,D,-1.158521,-2.317042


In [138]:
pivoted = df.pivot('date', 'variable')
pivoted

value                                  value2            \
variable           A         B         C         D         A         B   
date                                                                     
2000-01-03 -1.799869  2.488754  1.042697 -1.158521 -3.599738  4.977507   
2000-01-04  1.011323 -0.864634  0.413949 -0.043574  2.022646 -1.729269   
2000-01-05 -0.500127 -0.931677 -0.976139 -0.000943 -1.000253 -1.863354   

                                
variable           C         D  
date                            
2000-01-03  2.085394 -2.317042  
2000-01-04  0.827897 -0.087147  
2000-01-05 -1.952279 -0.001885

You of course can then select subsets from the `pivoted` DataFrame:

In [139]:
pivoted['value2']

variable,A,B,C,D
date,,,,
2000-01-03,-3.599738,4.977507,2.085394,-2.317042
2000-01-04,2.022646,-1.729269,0.827897,-0.087147
2000-01-05,-1.000253,-1.863354,-1.952279,-0.001885


Note that this returns a view on the underlying data in the case where the data are homogeneously-typed.

### Reshaping by stacking and unstacking

#### Stack and Unstack


Closely related to the `pivot` function are the related `stack` and `unstack` functions currently available on Series and DataFrame. These functions are designed to work together with `MultiIndex` objects (see the section on **hierarchical indexing**). Here are essentially what these functions do:
* `stack`: “pivot” a level of the (possibly hierarchical) column labels, returning a DataFrame with an index with a new inner-most level of row labels.
* `unstack`: inverse operation from `stack`: “pivot” a level of the (possibly hierarchical) row index to the column axis, producing a reshaped DataFrame with a new inner-most level of column labels.

The clearest way to explain is by example. Let’s take a prior example data set from the hierarchical indexing section:

In [140]:
tuples = list(zip(*[['bar', 'bar', 'baz', 'baz',
                     'foo', 'foo', 'qux', 'qux'],
                    ['one', 'two', 'one', 'two',
                     'one', 'two', 'one', 'two']]))

In [141]:
index = pd.MultiIndex.from_tuples(tuples, names=['first', 'second'])

In [142]:
df = pd.DataFrame(np.random.randn(8, 2), index=index, columns=['A', 'B'])

In [143]:
df2 = df[:4]
df

A         B
first second                    
bar   one     0.935313 -0.858013
      two    -0.307687  0.206815
baz   one     0.135774 -0.114830
      two    -0.644683  0.973409
foo   one     1.139032 -0.486812
      two    -0.223000  0.431308
qux   one     1.613028  1.823197
      two    -0.657459  0.036431

The `stack` function “compresses” a level in the DataFrame’s columns to produce either:
* A Series, in the case of a simple column Index
* A DataFrame, in the case of a `MultiIndex` in the columns

If the columns have a `MultiIndex`, you can choose which level to stack. The stacked level becomes the new lowest level in a `MultiIndex` on the columns:

In [144]:
df2

A         B
first second                    
bar   one     0.935313 -0.858013
      two    -0.307687  0.206815
baz   one     0.135774 -0.114830
      two    -0.644683  0.973409

In [145]:
stacked = df2.stack()
stacked
# lowest level is A,B
# second lowest first
# third lowest second

first  second   
bar    one     A    0.935313
               B   -0.858013
       two     A   -0.307687
               B    0.206815
baz    one     A    0.135774
               B   -0.114830
       two     A   -0.644683
               B    0.973409
dtype: float64

With a “stacked” DataFrame or Series (having a `MultiIndex` as the `index`), the inverse operation of `stack` is `unstack`, which by default unstacks the **last level**:

In [150]:
stacked

first  second   
bar    one     A    0.935313
               B   -0.858013
       two     A   -0.307687
               B    0.206815
baz    one     A    0.135774
               B   -0.114830
       two     A   -0.644683
               B    0.973409
dtype: float64

In [146]:
stacked.unstack()
# lowest level is A,B (-1)
# second lowest first (0)
# third lowest second (1)

A         B
first second                    
bar   one     0.935313 -0.858013
      two    -0.307687  0.206815
baz   one     0.135774 -0.114830
      two    -0.644683  0.973409

In [147]:
stacked.unstack().unstack()

A                   B          
second       one       two       one       two
first                                         
bar     0.935313 -0.307687 -0.858013  0.206815
baz     0.135774 -0.644683 -0.114830  0.973409

In [148]:
stacked.unstack(-1)

A         B
first second                    
bar   one     0.935313 -0.858013
      two    -0.307687  0.206815
baz   one     0.135774 -0.114830
      two    -0.644683  0.973409

In [151]:
stacked

first  second   
bar    one     A    0.935313
               B   -0.858013
       two     A   -0.307687
               B    0.206815
baz    one     A    0.135774
               B   -0.114830
       two     A   -0.644683
               B    0.973409
dtype: float64

In [149]:
stacked.unstack(0)
# lowest level is A,B (-1)
# second lowest first (0)
# third lowest second (1)

first          bar       baz
second                      
one    A  0.935313  0.135774
       B -0.858013 -0.114830
two    A -0.307687 -0.644683
       B  0.206815  0.973409

In [152]:
stacked.unstack(1)

second        one       two
first                      
bar   A  0.935313 -0.307687
      B -0.858013  0.206815
baz   A  0.135774 -0.644683
      B -0.114830  0.973409

If the indexes have names, you can **use the level names** instead of specifying the level numbers:

In [153]:
stacked.unstack('second')

second        one       two
first                      
bar   A  0.935313 -0.307687
      B -0.858013  0.206815
baz   A  0.135774 -0.644683
      B -0.114830  0.973409

Notice that the `stack` and `unstack` methods implicitly sort the index levels involved. Hence a call to `stack` and then `unstack`, or viceversa, will result in a **sorted** copy of the original DataFrame or Series:

In [154]:
index = pd.MultiIndex.from_product([[2,1], ['a', 'b']])

In [155]:
df = pd.DataFrame(np.random.randn(4), index=index, columns=['A'])
df

A
2 a -0.059809
  b  0.345969
1 a  1.190776
  b -0.024292

In [156]:
all(df.unstack().stack() == df.sort_index())

True

In [157]:
df.unstack().stack()

A
1 a  1.190776
  b -0.024292
2 a -0.059809
  b  0.345969

#### Multiple Levels


You may also stack or unstack more than one level at a time by passing a list of levels, in which case the end result is as if each level in the list were processed individually.

In [158]:
columns = pd.MultiIndex.from_tuples([('A', 'cat', 'long'), ('B', 'cat', 'long'),
                                     ('A', 'dog', 'short'), ('B', 'dog', 'short')],
                                    names=['exp', 'animal', 'hair_length'])

In [159]:
df = pd.DataFrame(np.random.randn(4, 4), columns=columns)
df

exp,A,B,A,B
animal,cat,cat,dog,dog
hair_length,long,long,short,short
0,1.114777,-1.281087,0.304165,-0.342239
1,0.629582,-0.382154,-0.161839,0.320521
2,1.419429,-1.932801,0.247321,0.080205
3,-0.762213,0.351397,0.231814,-0.879706


In [160]:
df.stack(level=['animal', 'hair_length'])

exp                          A         B
  animal hair_length                    
0 cat    long         1.114777 -1.281087
  dog    short        0.304165 -0.342239
1 cat    long         0.629582 -0.382154
  dog    short       -0.161839  0.320521
2 cat    long         1.419429 -1.932801
  dog    short        0.247321  0.080205
3 cat    long        -0.762213  0.351397
  dog    short        0.231814 -0.879706

The list of levels can contain either level names or level numbers (but not a mixture of the two).

In [161]:
# df.stack(level=['animal', 'hair_length'])
# from above is equivalent to:
df.stack(level=[1, 2])

exp                          A         B
  animal hair_length                    
0 cat    long         1.114777 -1.281087
  dog    short        0.304165 -0.342239
1 cat    long         0.629582 -0.382154
  dog    short       -0.161839  0.320521
2 cat    long         1.419429 -1.932801
  dog    short        0.247321  0.080205
3 cat    long        -0.762213  0.351397
  dog    short        0.231814 -0.879706

#### Missing Data


These functions are intelligent about handling missing data and do not expect each subgroup within the hierarchical index to have the same set of labels. They also can handle the index being unsorted (but you can make it sorted by calling `sort_index`, of course). Here is a more complex example:

In [162]:
columns = pd.MultiIndex.from_tuples([('A', 'cat'), ('B', 'dog'),
                                     ('B', 'cat'), ('A', 'dog')],
                                    names=['exp', 'animal'])

In [163]:
index = pd.MultiIndex.from_product([('bar', 'baz', 'foo', 'qux'),
                                    ('one', 'two')],
                                   names=['first', 'second'])

In [164]:
df = pd.DataFrame(np.random.randn(8, 4), index=index, columns=columns)

In [166]:
df2 = df.iloc[[0, 1, 2, 4, 5, 7]]
df2

exp                  A         B                   A
animal             cat       dog       cat       dog
first second                                        
bar   one     1.921505 -0.104017 -0.066122  1.015499
      two    -1.428860  1.614556  0.116519  1.058252
baz   one     1.509288 -0.986134  0.879134  0.104388
foo   one     0.413659 -0.908262 -0.225526 -0.860161
      two    -1.969212 -0.643794 -0.160660 -0.077712
qux   two     0.666575  0.786043 -1.369615  0.748888

In [167]:
df2.stack('exp')

animal                 cat       dog
first second exp                    
bar   one    A    1.921505  1.015499
             B   -0.066122 -0.104017
      two    A   -1.428860  1.058252
             B    0.116519  1.614556
baz   one    A    1.509288  0.104388
             B    0.879134 -0.986134
foo   one    A    0.413659 -0.860161
             B   -0.225526 -0.908262
      two    A   -1.969212 -0.077712
             B   -0.160660 -0.643794
qux   two    A    0.666575  0.748888
             B   -1.369615  0.786043

In [168]:
df2.stack('animal')

exp                         A         B
first second animal                    
bar   one    cat     1.921505 -0.066122
             dog     1.015499 -0.104017
      two    cat    -1.428860  0.116519
             dog     1.058252  1.614556
baz   one    cat     1.509288  0.879134
             dog     0.104388 -0.986134
foo   one    cat     0.413659 -0.225526
             dog    -0.860161 -0.908262
      two    cat    -1.969212 -0.160660
             dog    -0.077712 -0.643794
qux   two    cat     0.666575 -1.369615
             dog     0.748888  0.786043

Unstacking can result in missing values if subgroups do not have the same set of labels. By default, missing values will be replaced with the default fill value for that data type, `NaN` for `float`, `NaT` for `datetimelike`, etc. For integer types, by default data will converted to float and missing values will be set to `NaN`.

In [169]:
df3 = df.iloc[[0, 1, 4, 7], [1, 2]]
df3

exp                  B          
animal             dog       cat
first second                    
bar   one    -0.104017 -0.066122
      two     1.614556  0.116519
foo   one    -0.908262 -0.225526
qux   two     0.786043 -1.369615

In [170]:
df3.unstack()

exp            B                              
animal       dog                 cat          
second       one       two       one       two
first                                         
bar    -0.104017  1.614556 -0.066122  0.116519
foo    -0.908262       NaN -0.225526       NaN
qux          NaN  0.786043       NaN -1.369615

Alternatively, unstack takes an optional `fill_value` argument, for specifying the value of missing data.

### Reshaping by Melt


The `melt()` function is useful to massage a DataFrame into a format where one or more columns are **identifier variables**, while all other columns, considered *measured variables*, are “unpivoted” to the row axis, leaving just two non-identifier columns, “variable” and “value”. The names of those columns can be customized by supplying the `var_name` and `value_name` parameters.

In [171]:
cheese = pd.DataFrame({'first' : ['John', 'Mary'],
                       'last' : ['Doe', 'Bo'],
                       'height' : [5.5, 6.0],
                       'weight' : [130, 150]})
cheese

,first,height,last,weight
0,John,5.5,Doe,130
1,Mary,6.0,Bo,150


In [172]:
pd.melt(cheese, id_vars=['first', 'last'])

,first,last,variable,value
0,John,Doe,height,5.5
1,Mary,Bo,height,6.0
2,John,Doe,weight,130.0
3,Mary,Bo,weight,150.0


In [173]:
pd.melt(cheese, id_vars=['first', 'last'], var_name='quantity')

,first,last,quantity,value
0,John,Doe,height,5.5
1,Mary,Bo,height,6.0
2,John,Doe,weight,130.0
3,Mary,Bo,weight,150.0


### `Wide_to_long`

Another way to transform is to use the `wide_to_long` panel data convenience function.

In [174]:
dft = pd.DataFrame({"A1970" : {0 : "a", 1 : "b", 2 : "c"},
                    "A1980" : {0 : "d", 1 : "e", 2 : "f"},
                    "B1970" : {0 : 2.5, 1 : 1.2, 2 : .7},
                    "B1980" : {0 : 3.2, 1 : 1.3, 2 : .1},
                    "X"     : dict(zip(range(3), np.random.randn(3)))
                   })
dft

,A1970,A1980,B1970,B1980,X
0,a,d,2.5,3.2,-0.404530
1,b,e,1.2,1.3,-0.285303
2,c,f,0.7,0.1,0.017234


In [175]:
dft["id"] = dft.index
dft

,A1970,A1980,B1970,B1980,X,id
0,a,d,2.5,3.2,-0.404530,0
1,b,e,1.2,1.3,-0.285303,1
2,c,f,0.7,0.1,0.017234,2


In [176]:
pd.wide_to_long(dft, ["A", "B"], i="id", j="year")

,,X,A,B
id,year,,,
0,1970,-0.404530,a,2.5
1,1970,-0.285303,b,1.2
2,1970,0.017234,c,0.7
0,1980,-0.404530,d,3.2
1,1980,-0.285303,e,1.3
2,1980,0.017234,f,0.1


### Combining with stats and GroupBy [Advanced]

In [177]:
df

exp                  A         B                   A
animal             cat       dog       cat       dog
first second                                        
bar   one     1.921505 -0.104017 -0.066122  1.015499
      two    -1.428860  1.614556  0.116519  1.058252
baz   one     1.509288 -0.986134  0.879134  0.104388
      two     0.811085 -0.475306  0.140583  1.617311
foo   one     0.413659 -0.908262 -0.225526 -0.860161
      two    -1.969212 -0.643794 -0.160660 -0.077712
qux   one     0.171964  1.111495  0.281090  0.202382
      two     0.666575  0.786043 -1.369615  0.748888

In [178]:
df.stack()

exp                         A         B
first second animal                    
bar   one    cat     1.921505 -0.066122
             dog     1.015499 -0.104017
      two    cat    -1.428860  0.116519
             dog     1.058252  1.614556
baz   one    cat     1.509288  0.879134
             dog     0.104388 -0.986134
      two    cat     0.811085  0.140583
             dog     1.617311 -0.475306
foo   one    cat     0.413659 -0.225526
             dog    -0.860161 -0.908262
      two    cat    -1.969212 -0.160660
             dog    -0.077712 -0.643794
qux   one    cat     0.171964  0.281090
             dog     0.202382  1.111495
      two    cat     0.666575 -1.369615
             dog     0.748888  0.786043

In [179]:
df.stack().mean(1)

first  second  animal
bar    one     cat       0.927691
               dog       0.455741
       two     cat      -0.656170
               dog       1.336404
baz    one     cat       1.194211
               dog      -0.440873
       two     cat       0.475834
               dog       0.571003
foo    one     cat       0.094066
               dog      -0.884211
       two     cat      -1.064936
               dog      -0.360753
qux    one     cat       0.226527
               dog       0.656939
       two     cat      -0.351520
               dog       0.767466
dtype: float64

In [180]:
df.stack().mean(1).unstack()

animal             cat       dog
first second                    
bar   one     0.927691  0.455741
      two    -0.656170  1.336404
baz   one     1.194211 -0.440873
      two     0.475834  0.571003
foo   one     0.094066 -0.884211
      two    -1.064936 -0.360753
qux   one     0.226527  0.656939
      two    -0.351520  0.767466

In [181]:
df

exp                  A         B                   A
animal             cat       dog       cat       dog
first second                                        
bar   one     1.921505 -0.104017 -0.066122  1.015499
      two    -1.428860  1.614556  0.116519  1.058252
baz   one     1.509288 -0.986134  0.879134  0.104388
      two     0.811085 -0.475306  0.140583  1.617311
foo   one     0.413659 -0.908262 -0.225526 -0.860161
      two    -1.969212 -0.643794 -0.160660 -0.077712
qux   one     0.171964  1.111495  0.281090  0.202382
      two     0.666575  0.786043 -1.369615  0.748888

In [182]:
# same result, another way
df.groupby(level=1, axis=1).mean()

animal             cat       dog
first second                    
bar   one     0.927691  0.455741
      two    -0.656170  1.336404
baz   one     1.194211 -0.440873
      two     0.475834  0.571003
foo   one     0.094066 -0.884211
      two    -1.064936 -0.360753
qux   one     0.226527  0.656939
      two    -0.351520  0.767466

In [183]:
df.stack().groupby(level=1).mean()

exp,A,B
second,,
one,0.559815,-0.002293
two,0.178291,0.001041


In [184]:
df.mean().unstack(0)

exp,A,B
animal,,
cat,0.262000,-0.050575
dog,0.476106,0.049323


### Pivot tables


The function `pandas.pivot_table` can be used to create spreadsheet-style pivot tables. See the [cookbook](http://pandas.pydata.org/pandas-docs/stable/cookbook.html#cookbook-pivot) for some advanced strategies

It takes a number of arguments
* `data`: A DataFrame object
* `values`: a column or a list of columns to aggregate
* `index`: a column, Grouper, array which has the same length as data, or list of them. Keys to group by on the pivot table index. If an array is passed, it is being used as the same manner as column values.
* `columns`: a column, Grouper, array which has the same length as data, or list of them. Keys to group by on the pivot table column. If an array is passed, it is being used as the same manner as column values.
* `aggfunc`: function to use for aggregation, defaulting to `numpy.mean`

Consider a data set like this:

In [185]:
import datetime

In [186]:
df = pd.DataFrame({'A': ['one', 'one', 'two', 'three'] * 6,
                   'B': ['A', 'B', 'C'] * 8,
                   'C': ['foo', 'foo', 'foo', 'bar', 'bar', 'bar'] * 4,
                   'D': np.random.randn(24),
                   'E': np.random.randn(24),
                   'F': [datetime.datetime(2013, i, 1) for i in range(1, 13)] +
                   [datetime.datetime(2013, i, 15) for i in range(1, 13)]})
df

,A,B,C,D,E,F
0,one,A,foo,1.147293,-0.190384,2013-01-01
1,one,B,foo,-0.118580,0.817541,2013-02-01
2,two,C,foo,-0.682642,-1.046581,2013-03-01
3,three,A,bar,0.468574,0.946164,2013-04-01
4,one,B,bar,0.020654,-1.800023,2013-05-01
5,one,C,bar,-0.044312,0.419791,2013-06-01
6,two,A,foo,-0.244551,0.863929,2013-07-01
7,three,B,foo,-0.296644,0.138715,2013-08-01
8,one,C,foo,0.466439,-0.973733,2013-09-01
9,one,A,bar,1.002440,0.253338,2013-10-01


In [187]:
pd.pivot_table(df, values='D', index=['A', 'B'], columns=['C'])

C             bar       foo
A     B                    
one   A  0.300426  0.169518
      B  0.067093  1.142096
      C  0.075960 -0.543904
three A  0.180726       NaN
      B       NaN -0.242512
      C  0.384746       NaN
two   A       NaN  0.333085
      B  0.612433       NaN
      C       NaN -0.243428

In [188]:
pd.pivot_table(df, values='D', index=['B'], columns=['A', 'C'], aggfunc=np.sum)

A       one               three                 two          
C       bar       foo       bar       foo       bar       foo
B                                                            
A  0.600852  0.339036  0.361453       NaN       NaN  0.666170
B  0.134185  2.284191       NaN -0.485024  1.224867       NaN
C  0.151919 -1.087808  0.769492       NaN       NaN -0.486855

In [189]:
pd.pivot_table(df, values=['D','E'], index=['B'], columns=['A', 'C'], aggfunc=np.sum)

D                                                           E  \
A       one               three                 two                 one   
C       bar       foo       bar       foo       bar       foo       bar   
B                                                                         
A  0.600852  0.339036  0.361453       NaN       NaN  0.666170  0.372921   
B  0.134185  2.284191       NaN -0.485024  1.224867       NaN -1.464501   
C  0.151919 -1.087808  0.769492       NaN       NaN -0.486855  0.603775   

                                                     
A               three                 two            
C       foo       bar       foo       bar       foo  
B                                                    
A -0.378535  2.275341       NaN       NaN  0.374697  
B  1.484424       NaN  1.257079 -2.228075       NaN  
C -1.073044  0.257533       NaN       NaN -1.246494

The result object is a DataFrame having potentially hierarchical indexes on the rows and columns. If the `values` column name is not given, the pivot table will include all of the data that can be aggregated in an additional level of hierarchy in the columns:

In [190]:
pd.pivot_table(df, index=['A', 'B'], columns=['C'])

D                   E          
C             bar       foo       bar       foo
A     B                                        
one   A  0.300426  0.169518  0.186461 -0.189267
      B  0.067093  1.142096 -0.732251  0.742212
      C  0.075960 -0.543904  0.301887 -0.536522
three A  0.180726       NaN  1.137670       NaN
      B       NaN -0.242512       NaN  0.628540
      C  0.384746       NaN  0.128767       NaN
two   A       NaN  0.333085       NaN  0.187349
      B  0.612433       NaN -1.114037       NaN
      C       NaN -0.243428       NaN -0.623247

Also, you can use `Grouper` for `index` and `columns` keywords. For detail of `Grouper`, see *[Grouping with a Grouper specification](http://pandas.pydata.org/pandas-docs/stable/groupby.html#groupby-specify)*.

In [191]:
pd.pivot_table(df, values='D', index=pd.Grouper(freq='M', key='F'), columns='C')

C,bar,foo
F,,
2013-01-31,NaN,0.169518
2013-02-28,NaN,1.142096
2013-03-31,NaN,-0.243428
2013-04-30,0.180726,NaN
2013-05-31,0.067093,NaN
2013-06-30,0.075960,NaN
2013-07-31,NaN,0.333085
2013-08-31,NaN,-0.242512
2013-09-30,NaN,-0.543904


You can render a nice output of the table omitting the missing values by calling `to_string` if you wish:

In [192]:
table = pd.pivot_table(df, index=['A', 'B'], columns=['C'])
print(table.to_string(na_rep=''))

                D                   E          
C             bar       foo       bar       foo
A     B                                        
one   A  0.300426  0.169518  0.186461 -0.189267
      B  0.067093  1.142096 -0.732251  0.742212
      C  0.075960 -0.543904  0.301887 -0.536522
three A  0.180726            1.137670          
      B           -0.242512            0.628540
      C  0.384746            0.128767          
two   A            0.333085            0.187349
      B  0.612433           -1.114037          
      C           -0.243428           -0.623247


Note that `pivot_table` is also available as an instance method on DataFrame.

#### Adding margins


If you pass `margins=True` to `pivot_table`, special All columns and rows will be added with partial group aggregates across the categories on the rows and columns:

In [193]:
df.pivot_table(index=['A', 'B'], columns='C', margins=True, aggfunc=np.std)

D                             E                    
C             bar       foo       All       bar       foo       All
A     B                                                            
one   A  0.992798  1.382783  0.985710  0.094579  0.001579  0.223696
      B  0.065674  1.782865  1.202574  1.510058  0.106532  1.220063
      C  0.170090  1.428841  0.904571  0.166740  0.618310  0.609108
three A  0.407077       NaN  0.407077  0.270831       NaN  0.270831
      B       NaN  0.076554  0.076554       NaN  0.692717  0.692717
      C  0.027320       NaN  0.027320  0.894035       NaN  0.894035
two   A       NaN  0.816901  0.816901       NaN  0.956829  0.956829
      B  0.427223       NaN  0.427223  1.177564       NaN  1.177564
      C       NaN  0.621143  0.621143       NaN  0.598685  0.598685
All      0.404987  1.034900  0.757029  1.000074  0.708340  0.830058

### Cross tabulations


Use the crosstab function to compute a cross-tabulation of two (or more) factors. By default `crosstab` computes a frequency table of the factors unless an array of values and an aggregation function are passed.

It takes a number of arguments

* `index`: array-like, values to group by in the rows
* `columns`: array-like, values to group by in the columns
* `values`: array-like, optional, array of values to aggregate according to the factors
* `aggfunc`: function, optional, If no values array is passed, computes a frequency table
* `rownames`: sequence, default None, must match number of row arrays passed
* `colnames`: sequence, default None, if passed, must match number of column arrays passed
* `margins`: boolean, default False, Add row/column margins (subtotals)
* `normalize`: boolean, {‘all’, ‘index’, ‘columns’}, or {0,1}, default False. Normalize by dividing all values by the sum of values.

Any Series passed will have their name attributes used unless row or column names for the cross-tabulation are specified

For example:

In [195]:
foo, bar, dull, shiny, one, two = 'foo', 'bar', 'dull', 'shiny', 'one', 'two'
a = np.array([foo, foo, bar, bar, foo, foo], dtype=object)
b = np.array([one, one, two, one, two, one], dtype=object)
c = np.array([dull, dull, shiny, dull, dull, shiny], dtype=object)

In [196]:
a

array(['foo', 'foo', 'bar', 'bar', 'foo', 'foo'], dtype=object)

In [197]:
b

array(['one', 'one', 'two', 'one', 'two', 'one'], dtype=object)

In [198]:
c

array(['dull', 'dull', 'shiny', 'dull', 'dull', 'shiny'], dtype=object)

In [199]:
pd.crosstab(a, [b, c], rownames=['a'], colnames=['b', 'c'])

b    one        two      
c   dull shiny dull shiny
a                        
bar    1     0    0     1
foo    2     1    1     0

If `crosstab` receives only two Series, it will provide a frequency table.

In [200]:
df = pd.DataFrame({'A': [1, 2, 2, 2, 2], 'B': [3, 3, 4, 4, 4],
                   'C': [1, 1, np.nan, 1, 1]})
df

,A,B,C
0,1,3,1.0
1,2,3,1.0
2,2,4,NaN
3,2,4,1.0
4,2,4,1.0


In [201]:
pd.crosstab(df.A, df.B)

B,3,4
A,,
1,1,0
2,1,3


#### Normalisation


Frequency tables can also be normalized to show percentages rather than counts using the normalize argument:

In [202]:
pd.crosstab(df.A, df.B, normalize=True)

B,3,4
A,,
1,0.2,0.0
2,0.2,0.6


In [203]:
pd.crosstab(df.A, df.B, normalize='columns')

B,3,4
A,,
1,0.5,0.0
2,0.5,1.0


In [204]:
pd.crosstab(df.A, df.B, values=df.C, aggfunc=np.sum)

B,3,4
A,,
1,1.0,NaN
2,1.0,2.0


In [205]:
pd.crosstab(df.A, df.B, values=df.C, aggfunc=np.sum, normalize=True,
            margins=True)

B,3,4,All
A,,,
1,0.25,0.0,0.25
2,0.25,0.5,0.75
All,0.50,0.5,1.00


### Tiling


The `cut` function computes groupings for the values of the input array and is often used to transform continuous variables to discrete or categorical variables:

In [206]:
ages = np.array([10, 15, 13, 12, 23, 25, 28, 59, 60])
pd.cut(ages, bins=3)

[(9.95, 26.667], (9.95, 26.667], (9.95, 26.667], (9.95, 26.667], (9.95, 26.667], (9.95, 26.667], (26.667, 43.333], (43.333, 60.0], (43.333, 60.0]]
Categories (3, interval[float64]): [(9.95, 26.667] < (26.667, 43.333] < (43.333, 60.0]]

In [208]:
pd.Series(pd.cut(ages, bins=3))

0      (9.95, 26.667]
1      (9.95, 26.667]
2      (9.95, 26.667]
3      (9.95, 26.667]
4      (9.95, 26.667]
5      (9.95, 26.667]
6    (26.667, 43.333]
7      (43.333, 60.0]
8      (43.333, 60.0]
dtype: category
Categories (3, interval[float64]): [(9.95, 26.667] < (26.667, 43.333] < (43.333, 60.0]]

### Computing indicator / dummy variables


To convert a categorical variable into a “dummy” or “indicator” DataFrame, for example a column in a DataFrame (a Series) which has `k` distinct values, can derive a DataFrame containing `k` columns of 1s and 0s:

In [209]:
df = pd.DataFrame({'key': list('bbacab'), 'data1': range(6)})
df

,data1,key
0,0,b
1,1,b
2,2,a
3,3,c
4,4,a
5,5,b


In [210]:
pd.get_dummies(df['key'])

,a,b,c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


Sometimes it’s useful to prefix the column names, for example when merging the result with the original DataFrame:

In [211]:
dummies = pd.get_dummies(df['key'], prefix='key')
dummies

,key_a,key_b,key_c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


In [212]:
df[['data1']].join(dummies)

,data1,key_a,key_b,key_c
0,0,0,1,0
1,1,0,1,0
2,2,1,0,0
3,3,0,0,1
4,4,1,0,0
5,5,0,1,0


This function is often used along with discretization functions like cut:

In [213]:
values = np.random.randn(10)
values

array([-2.03221632, -0.67226415,  0.41857761, -1.40311592,  0.59911205,
        0.90197634,  0.36597857,  0.85119731,  0.16013478,  0.05267345])

In [214]:
bins = [0, 0.2, 0.4, 0.6, 0.8, 1]
pd.get_dummies(pd.cut(values, bins))

,"(0.0, 0.2]","(0.2, 0.4]","(0.4, 0.6]","(0.6, 0.8]","(0.8, 1.0]"
0,0,0,0,0,0
1,0,0,0,0,0
2,0,0,1,0,0
3,0,0,0,0,0
4,0,0,1,0,0
5,0,0,0,0,1
6,0,1,0,0,0
7,0,0,0,0,1
8,1,0,0,0,0
9,1,0,0,0,0


**get_dummies()** also accepts a DataFrame. By default all categorical variables (categorical in the statistical sense, those with *object* or *categorical* dtype) are encoded as dummy variables.

In [215]:
df = pd.DataFrame({'A': ['a', 'b', 'a'], 'B': ['c', 'c', 'b'],
                   'C': [1, 2, 3]})

In [216]:
pd.get_dummies(df)

,C,A_a,A_b,B_b,B_c
0,1,1,0,0,1
1,2,0,1,0,1
2,3,1,0,1,0


All non-object columns are included untouched in the output.

You can control the columns that are encoded with the `columns` keyword.

In [218]:
df

,A,B,C
0,a,c,1
1,b,c,2
2,a,b,3


In [217]:
pd.get_dummies(df, columns=['A'])

,B,C,A_a,A_b
0,c,1,1,0
1,c,2,0,1
2,b,3,1,0


Notice that the `B` column is still included in the output, it just hasn’t been encoded. You can drop `B` before calling `get_dummies` if you don’t want to include it in the output.

As with the Series version, you can pass values for the `prefix` and `prefix_sep`. By default the column name is used as the prefix, and ‘_’ as the prefix separator. You can specify `prefix` and `prefix_sep` in 3 ways

* string: Use the same value for `prefix` or `prefix_sep` for each column to be encoded
* list: Must be the same length as the number of columns being encoded.
* dict: Mapping column name to prefix

In [219]:
simple = pd.get_dummies(df, prefix='new_prefix')
simple

,C,new_prefix_a,new_prefix_b,new_prefix_b,new_prefix_c
0,1,1,0,0,1
1,2,0,1,0,1
2,3,1,0,1,0


In [220]:
from_list = pd.get_dummies(df, prefix=['from_A', 'from_B'])
from_list

,C,from_A_a,from_A_b,from_B_b,from_B_c
0,1,1,0,0,1
1,2,0,1,0,1
2,3,1,0,1,0


In [221]:
from_dict = pd.get_dummies(df, prefix={'B': 'from_B', 'A': 'from_A'})
from_dict

,C,from_A_a,from_A_b,from_B_b,from_B_c
0,1,1,0,0,1
1,2,0,1,0,1
2,3,1,0,1,0


Sometimes it will be useful to only keep `k-1` levels of a categorical variable to avoid collinearity when feeding the result to statistical models. You can switch to this mode by turn on `drop_first`.

In [222]:
s = pd.Series(list('abcaa'))

In [223]:
pd.get_dummies(s)

,a,b,c
0,1,0,0
1,0,1,0
2,0,0,1
3,1,0,0
4,1,0,0


In [224]:
pd.get_dummies(s, drop_first=True)

,b,c
0,0,0
1,1,0
2,0,1
3,0,0
4,0,0


When a column contains only one level, it will be omitted in the result.

In [225]:
df = pd.DataFrame({'A':list('aaaaa'),'B':list('ababc')})

In [226]:
pd.get_dummies(df)

,A_a,B_a,B_b,B_c
0,1,1,0,0
1,1,0,1,0
2,1,1,0,0
3,1,0,1,0
4,1,0,0,1


In [227]:
pd.get_dummies(df, drop_first=True)

,B_b,B_c
0,0,0
1,1,0
2,0,0
3,1,0
4,0,1


***